# 📊 Data Processing & Analysis Using Code Interpreter

Welcome to the Data Processing & Analysis module using AWS Bedrock AgentCore Code Interpreter! This notebook focuses on statistical calculations and data manipulation with simple, clear examples.

## 🎯 Learning Objectives

In this notebook, you'll learn how to:

- 🔧 Set up and use AgentCore Code Interpreter with both boto3 and AgentCore SDK
- 📈 Perform basic statistical calculations on simple datasets
- 🔄 Process and analyze data using both JavaScript (Deno) and Python
- 🏗️ Build framework-agnostic abstractions for data analysis

## 📋 Prerequisites

- AWS account with access to Amazon Bedrock AgentCore
- Proper IAM permissions for AgentCore services
- Python 3.12 environment with required dependencies

Let's begin! 🚀

## Amazon Bedrock AgentCore Code Interpreter


The Amazon Bedrock AgentCore Code Interpreter enables AI agents to write and execute code securely in sandbox environments, enhancing their accuracy and expanding their ability to solve complex end-to-end tasks. This is critical in Agentic AI applications where the agents may execute arbitrary code that can lead to data compromise or security risks. The AgentCore Code Interpreter tool provides secure code execution, which helps you avoid running into these issues.

The Code Interpreter comes with pre-built runtimes for multiple languages and advanced features, including large file support and internet access, and CloudTrail logging capabilities. For inline upload, the file size can be up to 100 MB. And for uploading to Amazon S3 through terminal commands, the file size can be as large as 5 GB.

Developers can customize environments with session properties and network modes to meet their enterprise and security requirements. The AgentCore Code Interpreter reduces manual intervention while enabling sophisticated AI development without compromising security or performance.

![Amazon Bedrock AgentCore Code Interpreter Architecture](../../../media/agent_core_code_interpreter_arch.png)

## 🔧 Section 1: Environment Setup

In [ ]:
# 🔧 Environment Setup Instructions

# IMPORTANT: This notebook requires Python 3.12.8 as specified in pyproject.toml
# All dependencies are managed via UV (recommended approach for this project)

# SETUP STEPS:
# 1. Create virtual environment: uv venv
# 2. Install all dependencies: uv sync
# 3. Launch Jupyter Lab: uv run jupyter lab

# FOR VS CODE USERS:
# 1. Run: uv run python -c "import sys; print(sys.executable)"
# 2. Copy the path and select it as your Python interpreter in VS Code
# 3. The kernel should now have access to all required dependencies

# Verify environment setup
import sys
print(f"✅ Python version: {sys.version}")
print(f"✅ Python executable: {sys.executable}")
print("✅ Environment setup complete! All dependencies should be available via UV.")

In [ ]:
# Import core dependencies
import boto3
import json
import os
import time
from typing import Dict, Any, List, Optional

# Import AgentCore SDK
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter, code_session

# Import existing Module 3 components
from agentic_platform.core.models.memory_models import Message, SessionContext
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse
from agentic_platform.core.models.api_models import AgenticRequest, AgenticResponse

print("✅ Dependencies imported successfully!")

In [ ]:
# Initialize AWS clients with specific profile
profile_name = "XXXXXXX" # your aws profile name
session = boto3.Session(profile_name=profile_name)
region_name = "us-west-2"

# Verify credentials are working
try:
    credentials = session.get_credentials()
    print(f"✅ Using AWS profile: {profile_name}")
    print(f"✅ Access Key: {credentials.access_key[:10]}...")
    print(f"✅ Region: {region_name}")
except Exception as e:
    print(f"❌ Error with AWS credentials: {e}")
    raise

# Standard AgentCore client
bedrock_agentcore_client = session.client(
    'bedrock-agentcore',
    region_name=region_name,
    endpoint_url=f"https://bedrock-agentcore.{region_name}.amazonaws.com"
)

print("✅ AWS clients initialized successfully with profile!")

## 📊 Section 2: Simple Data Processing with Boto3

Let's start with a simple example using JavaScript  to process temperature data.

In [ ]:
# Simple JavaScript code for basic statistical analysis
# NOTE: Emojis removed from JavaScript code to avoid Deno parsing errors
javascript_stats_code = """
// Simple temperature data analysis example
console.log('Starting simple data analysis...');

// Small dataset: Daily temperatures in Celsius
const temperatures = [22.5, 24.1, 23.8, 25.2, 21.9];
const days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'];

// Calculate basic statistics
function calculateBasicStats(data) {
    // Calculate sum
    const sum = data.reduce((a, b) => a + b, 0);
    
    // Calculate mean
    const mean = sum / data.length;
    
    // Find min and max
    const min = Math.min(...data);
    const max = Math.max(...data);
    
    // Calculate variance
    const variance = data.reduce((acc, val) => {
        return acc + Math.pow(val - mean, 2);
    }, 0) / data.length;
    
    return {
        count: data.length,
        sum: sum.toFixed(2),
        mean: mean.toFixed(2),
        min: min.toFixed(2),
        max: max.toFixed(2),
        variance: variance.toFixed(2),
        range: (max - min).toFixed(2)
    };
}

// Perform analysis
const stats = calculateBasicStats(temperatures);

// Display results
console.log('=====================================');
console.log('Number of readings: ' + stats.count);
console.log('Average temperature: ' + stats.mean + ' C');
console.log('Minimum temperature: ' + stats.min + ' C');
console.log('Maximum temperature: ' + stats.max + ' C');
console.log('Temperature range: ' + stats.range + ' C');
console.log('Variance: ' + stats.variance);

// Display daily readings
console.log('=== DAILY READINGS ===');
days.forEach((day, index) => {
    console.log(day + ': ' + temperatures[index] + ' C');
});

console.log('=====================================');
console.log('Analysis complete!');

// Return the statistics object
stats;
"""

print("JavaScript statistical analysis code prepared!")

In [ ]:
def execute_code_with_boto3(code: str, language: str = "javascript") -> Dict[str, Any]:
    """
    Execute code using boto3 direct API calls to AgentCore Code Interpreter.
    
    Args:
        code: The code to execute
        language: Programming language (javascript or python)
    
    Returns:
        Dictionary with execution results
    """
    try:
        # Start a code interpreter session
        session_response = bedrock_agentcore_client.start_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            name="boto3-stats-session",
            sessionTimeoutSeconds=300
        )
        session_id = session_response["sessionId"]
        print(f"✅ Started session: {session_id}")
        
        # Execute the code
        execute_response = bedrock_agentcore_client.invoke_code_interpreter(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id,
            name="executeCode",
            arguments={
                "language": language,
                "code": code
            }
        )
        
        # Process the response stream
        results = []
        for event in execute_response['stream']:
            if 'result' in event:
                result = event['result']
                results.append(result)
                
                # Print text output
                if 'content' in result:
                    for content_item in result['content']:
                        if content_item['type'] == 'text':
                            print(f"Output: {content_item['text']}")
        
        # Clean up session
        bedrock_agentcore_client.stop_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id
        )
        print(f"✅ Session {session_id} stopped")
        
        return {
            "success": True,
            "session_id": session_id,
            "results": results
        }
        
    except Exception as e:
        print(f"❌ Error executing code: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }

In [ ]:
# Execute the JavaScript statistical analysis code using boto3
print("🚀 Executing JavaScript statistical analysis with boto3...\n")
boto3_result = execute_code_with_boto3(javascript_stats_code, "javascript")
print(f"\n✅ Execution completed: {boto3_result['success']}")

## 🐍 Section 3: Simple Data Processing with AgentCore SDK

Now let's perform the same analysis using Python through the AgentCore SDK.

In [ ]:
# Simple Python code for basic statistical analysis
python_stats_code = """
import statistics

# Simple dataset: Student test scores
print('Starting simple data analysis...')

scores = [85, 92, 78, 95, 88]
students = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']

# Calculate basic statistics
def analyze_scores(data):
    stats = {
        'count': len(data),
        'sum': sum(data),
        'mean': statistics.mean(data),
        'median': statistics.median(data),
        'min': min(data),
        'max': max(data),
        'range': max(data) - min(data),
        'variance': statistics.variance(data),
        'stdev': statistics.stdev(data)
    }
    return stats

# Perform analysis
results = analyze_scores(scores)

# Display results
print('=== TEST SCORE ANALYSIS RESULTS ===')
print('=' * 35)
print(f'Number of students: {results["count"]}')
print(f'Average score: {results["mean"]:.1f}')
print(f'Median score: {results["median"]:.1f}')
print(f'Lowest score: {results["min"]}')
print(f'Highest score: {results["max"]}')
print(f'Score range: {results["range"]}')
print(f'Variance: {results["variance"]:.2f}')
print(f'Standard deviation: {results["stdev"]:.2f}')

# Display individual scores
print('=== INDIVIDUAL SCORES ===')
for student, score in zip(students, scores):
    grade = 'A' if score >= 90 else 'B' if score >= 80 else 'C'
    print(f'{student}: {score} (Grade: {grade})')

# Identify top performers
threshold = results['mean']
above_average = [(students[i], scores[i]) for i in range(len(scores)) if scores[i] > threshold]
print(f'=== Above average students ({threshold:.1f}+) ===')
for student, score in above_average:
    print(f'  - {student}: {score}')

print('Analysis complete!')

# Return the results
results
"""

print("Python statistical analysis code prepared!")

In [ ]:
# Alternative: If the above doesn't work, try using environment variables
def execute_code_with_sdk(code: str, language: str = "python") -> Dict[str, Any]:
    """
    Alternative approach: Execute code using the AgentCore SDK with credentials from environment.
    
    Args:
        code: The code to execute
        language: Programming language (javascript or python)
    
    Returns:
        Dictionary with execution results
    """
    try:
        # Get credentials from the session
        credentials = session.get_credentials()
        region_name = "us-west-2"
        # Set environment variables for AWS SDK
        import os
        os.environ['AWS_ACCESS_KEY_ID'] = credentials.access_key
        os.environ['AWS_SECRET_ACCESS_KEY'] = credentials.secret_key
        if credentials.token:
            os.environ['AWS_SESSION_TOKEN'] = credentials.token
        os.environ['AWS_DEFAULT_REGION'] = region_name
        
        print("✅ AWS credentials set in environment variables")
        
        # Now create the CodeInterpreter client
        code_client = CodeInterpreter("us-west-2")
        code_client.start()
        print("✅ AgentCore SDK session started with environment credentials")
        
        # Execute the code
        response = code_client.invoke("executeCode", {
            "language": language,
            "code": code
        })
        
        # Process and print the response
        results = []
        for event in response["stream"]:
            if "result" in event:
                result = event["result"]
                results.append(result)
                
                # Extract and display text content
                if 'content' in result:
                    for content_item in result['content']:
                        if content_item['type'] == 'text':
                            print(content_item['text'])
        
        # Clean up and stop the session
        code_client.stop()
        print("\n✅ AgentCore SDK session stopped")
        
        return {
            "success": True,
            "results": results
        }
        
    except Exception as e:
        print(f"❌ Error executing code with SDK: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }
    finally:
        # Clean up environment variables
        for key in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN']:
            if key in os.environ:
                del os.environ[key]

In [ ]:
# Execute the Python statistical analysis code using AgentCore SDK
print("🚀 Executing Python statistical analysis with AgentCore SDK...\n")
sdk_result = execute_code_with_sdk(python_stats_code, "python")
print(f"\n✅ Execution completed: {sdk_result['success']}")

## Conclusion

In this notebook, we've successfully demonstrated:

1. **Execute Code sample on a code interpreter sandbox**: 
   - Calculating mean, median, min, max, and variance
   - Processing simple datasets efficiently

2. **Two Implementation Approaches**:
   - **Boto3**: Direct API calls for fine-grained control
   - **AgentCore SDK**: Higher-level abstraction for simpler code

3. **Multi-language Support**:
   - JavaScript for server-side processing
   - Python for data analysis with built-in statistics module


### 📚 Additional Resources

- [AWS Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock-agentcore/)
- [AgentCore Code Interpreter API Reference](https://docs.aws.amazon.com/bedrock-agentcore/latest/APIReference/)
- [Module 3 Other Notebooks](../README.md)

